In [2]:
# docker

# LangChain notebook 

LangChain is a framework for developing applications powered by language models.

- GitHub: https://github.com/hwchase17/langchain
- Docs: https://python.langchain.com/v0.2/docs/introduction/

### Overview:
- Installation
- LLMs
- Prompt Templates
- Chains
- Agents and Tools
- Memory
- Document Loaders
- Indexes

In [3]:
%pip install langchain langchain_community -q

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install dotenv -q

Note: you may need to restart the kernel to use updated packages.


In [22]:
from dotenv import load_dotenv

openai_key = load_dotenv("OPENAI_API_KEY")
hf_token = load_dotenv("HUGGINGFACEHUB_API_TOKEN")

## LLMs in LangChain

The basic building block of LangChain is a Large Language Model which takes text as input and generates more text.

Suppose we want to generate a company name based on the company description, so we will first initialize an OpenAI wrapper. In this case, since we want the output to be more random, we will intialize our model with high temprature.

The temperature parameter adjusts the randomness of the output. Higher values like 0.7 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.

temperature value--> how creative we want our model to be

0 ---> temperature it means model is  very safe it is not taking any bets.

1 --> it will take risk it might generate wrong output but it is very creative

In [6]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)

/tmp/ipykernel_22339/166103413.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.9)


In [7]:
text="what would be a good company name for a company that make colorful socks?"

In [8]:
print(llm.model_name)   # defaults to gpt 3.5 turbo instruct if none is selected
print(llm.predict(text))

gpt-3.5-turbo-instruct


/tmp/ipykernel_22339/229049954.py:2: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm.predict(text))




Happy Feet Socks Co.


In [9]:
# equivalent
print(llm(text))

/tmp/ipykernel_22339/4203956707.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm(text))




"VibrantKnits" or "RainbowSocks Co."


In [10]:
# equivalent 
print(llm.invoke(text))



"Rainbow Footwear Co." or "Vibrant Socks Co."


## LangChain and HuggingFace

Hugging Face allows us to use open source models:

In [12]:
%pip install huggingface_hub -q

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain import HuggingFaceHub

In [ ]:
llm = HuggingFaceHub(repo_id="google/flan-t5-large", 
                     model_kwargs={"temperature":0, "max_length":64},
                     huggingfacehub_api_token=hf_token)

llm("translate English to German: How old are you?")

ValidationError: 1 validation error for HuggingFaceHub
  Value error, Did not find huggingfacehub_api_token, please add an environment variable `HUGGINGFACEHUB_API_TOKEN` which contains it, or pass `huggingfacehub_api_token` as a named parameter. [type=value_error, input_value={'repo_id': 'google/flan-...acehub_api_token': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

## Prompt Templates